# ETL Pipeline
To load data from SQL into python

In [1]:
!pip install sqlalchemy
!pip install dask
!pip install pyodbc

In [2]:
import os
import pandas as pd
import dask.dataframe as dd
from sqlalchemy import create_engine

from warnings import filterwarnings 
filterwarnings('ignore')

# Extract
Load all 3 tables from SQL into pandas DataFrames

In [3]:
%%time

# set to your own desktop name
pc_name = os.environ['COMPUTERNAME']

# connect to MS-SQL
server = f"{pc_name}\SQLEXPRESS" # SQL Server Name
database = "JustTaxi" # database name
con_string = f'mssql+pyodbc://{server}/{database}?driver=SQL Server'
engine = create_engine(con_string)

# retrieve data
connection = engine.connect()

# driver data
drivers = connection.execute('SELECT * FROM drivers')
driver_data = pd.DataFrame(data=drivers.fetchall(), columns=drivers.keys())

# trip data
trips = connection.execute('SELECT * FROM trips')
trip_data = pd.DataFrame(data=trips.fetchall(), columns=trips.keys())

connection.close() # close connection explicitly

Wall time: 323 ms


In [4]:
%%time
# get sensor data by chunksize
# connection = engine.connect().execution_options(stream_results=True)
sensor_data_generator = pd.read_sql_query('SELECT * FROM sensor_data', con_string, chunksize=10**5)
sensor_data = pd.concat([chunk for chunk in sensor_data_generator])

Wall time: 4min 20s


In [5]:
driver_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   driver_id      148 non-null    object
 1   name           148 non-null    object
 2   date_of_birth  148 non-null    object
 3   gender         148 non-null    object
 4   car_model      148 non-null    object
 5   car_make_year  148 non-null    object
 6   rating         148 non-null    object
dtypes: object(7)
memory usage: 8.2+ KB


In [6]:
trip_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   booking_id    20000 non-null  object
 1   driver_id     20000 non-null  object
 2   safety_label  20000 non-null  object
dtypes: object(3)
memory usage: 468.9+ KB


In [7]:
sensor_data.info()  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7469656 entries, 0 to 69655
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   booking_id      object 
 1   accuracy        float64
 2   bearing         float64
 3   acceleration_x  float64
 4   acceleration_y  float64
 5   acceleration_z  float64
 6   gyro_x          float64
 7   gyro_y          float64
 8   gyro_z          float64
 9   second          float64
 10  speed           float64
dtypes: float64(10), object(1)
memory usage: 683.9+ MB


# Transform
Clean and merge the DataFrames

## Remove duplicated data

In [8]:
# remove duplicated data from all 3 dataframes
driver_data = driver_data.drop_duplicates()
trip_data = trip_data.drop_duplicates()
sensor_data = sensor_data.drop_duplicates()

## Merge dataframes

In [9]:
%%time

# merge driver and trip data
driver_trips = trip_data.merge(driver_data, on='driver_id', how='left')

# merge driver_trips and sensor data
driver_trips_sensor = sensor_data.merge(driver_trips, on='booking_id', how='left')

Wall time: 1.75 s


In [10]:
driver_trips_sensor.head()

,booking_id,accuracy,bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,speed,driver_id,safety_label,name,date_of_birth,gender,car_model,car_make_year,rating
0,0,8.0,143.298294,-1.706207,-9.270792,-1.209448,-0.028965,-0.032652,0.015390,2.0,0.228454,48,0,Lilia,1974-08-13,Male,Mercedes-Benz,2013-01-01,4.0
1,0,8.0,143.298294,-1.416705,-9.548032,-1.860977,-0.022413,0.005049,-0.025753,3.0,0.228454,48,0,Lilia,1974-08-13,Male,Mercedes-Benz,2013-01-01,4.0
2,0,8.0,143.298294,-0.346924,-9.532629,-1.204663,0.014962,-0.050033,0.025118,9.0,0.228454,48,0,Lilia,1974-08-13,Male,Mercedes-Benz,2013-01-01,4.0
3,0,8.0,143.298294,-0.600986,-9.452029,-2.157507,0.004548,-0.011713,-0.004078,11.0,0.228454,48,0,Lilia,1974-08-13,Male,Mercedes-Benz,2013-01-01,4.0
4,0,8.0,143.298294,-0.597546,-9.863403,-1.672711,-0.000401,0.000315,-0.009830,12.0,0.228454,48,0,Lilia,1974-08-13,Male,Mercedes-Benz,2013-01-01,4.0


In [11]:
driver_trips_sensor.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7469656 entries, 0 to 7469655
Data columns (total 19 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   booking_id      7469656 non-null  object 
 1   accuracy        7351631 non-null  float64
 2   bearing         7350253 non-null  float64
 3   acceleration_x  7364539 non-null  float64
 4   acceleration_y  7346553 non-null  float64
 5   acceleration_z  7370520 non-null  float64
 6   gyro_x          7336398 non-null  float64
 7   gyro_y          7329944 non-null  float64
 8   gyro_z          7404168 non-null  float64
 9   second          7469656 non-null  float64
 10  speed           7346079 non-null  float64
 11  driver_id       7469656 non-null  object 
 12  safety_label    7469656 non-null  object 
 13  name            7469656 non-null  object 
 14  date_of_birth   7469656 non-null  object 
 15  gender          7469656 non-null  object 
 16  car_model       7469656 non-null  ob

In [12]:
taxi_data = driver_trips_sensor[['booking_id', 'driver_id', 'name', 'date_of_birth', 'gender', 'car_model', 'car_make_year', 'accuracy', 'bearing', 'acceleration_x', 'acceleration_y', 'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z', 'second', 'speed', 'rating', 'safety_label']]
taxi_data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7469656 entries, 0 to 7469655
Data columns (total 19 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   booking_id      7469656 non-null  object 
 1   driver_id       7469656 non-null  object 
 2   name            7469656 non-null  object 
 3   date_of_birth   7469656 non-null  object 
 4   gender          7469656 non-null  object 
 5   car_model       7469656 non-null  object 
 6   car_make_year   7469656 non-null  object 
 7   accuracy        7351631 non-null  float64
 8   bearing         7350253 non-null  float64
 9   acceleration_x  7364539 non-null  float64
 10  acceleration_y  7346553 non-null  float64
 11  acceleration_z  7370520 non-null  float64
 12  gyro_x          7336398 non-null  float64
 13  gyro_y          7329944 non-null  float64
 14  gyro_z          7404168 non-null  float64
 15  second          7469656 non-null  float64
 16  speed           7346079 non-null  fl

In [13]:
%%time
# if directory does not exist, create it
if not os.path.exists('../data/cleaned/'):
    os.makedirs('../data/cleaned/')

# save data to csv
taxi_data.to_csv('../data/cleaned/taxi_data.csv', index=False)

Wall time: 1min 17s
